In [ ]:

#Importing Packages

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy.random as nr
import scipy.stats as ss
import math
import statsmodels.stats.weightstats as ws
from statsmodels.stats.power import tt_ind_solve_power
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from sklearn import preprocessing
import sklearn.model_selection as ms
from sklearn import linear_model, mixture
import sklearn.metrics as sklm
from sklearn.ensemble import AdaBoostRegressor
from sklearn.cluster import KMeans, AgglomerativeClustering, SpectralClustering, DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

import plotly.graph_objs as go
import matplotlib.pyplot as plt
import seaborn as sns


import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
#import basic libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [ ]:
#Importing the Data
ins = pd.read_csv('../input/insurance/insurance.csv')
ins.head()

About the Data
age: Age of primary beneficiary.
sex: Insurance contractor gender: female, male.
bmi: Body mass index, providing an understanding of body, weights that are relatively high or low relative to height, objective index of body weight (kg / m ^ 2) using the ratio of height to weight, ideally 18.5 to 24.9.
children: Number of children covered by health insurance / Number of dependents.
smoker: Whether the contractor is a smoker or not. .
region: The beneficiary's residential area in the US, northeast, southeast, southwest, northwest.
charges: Individual medical costs billed by health insurance.

 along with the other interesting insights we gained form this table, I hope that this data will be a warning for smokers, and non-smokers, about the hazardous nature of this substance.
 
the problem: is the increased diseases that caused the smoking.
 solutions:I hope that this data will be a warning for smokers, and non-smokers, about the hazardous nature of this substance.
Procedures:
1- Visualizing and inspect the Data to get as much an insight about it as we can.
2- Statistical analysis of data
3- Model data either classification

firstly i will provide the Visualize concept into this data:

In [ ]:
ins.hist(layout = (3, 3), figsize=(12, 9), color='blue', grid=False, bins=15)

Histogram Observations:
-We can see that only the bmi feature is normally distributed, with a mean slightly above the maximum accepted value = 30.
-Age seems to be uniformly distributed, except for the young ages at the far left, where we have more data coming from this age group.
-Charges and children features are right-skewed.
-For children, this is expected, as people prefer to have few (or no children) these days than to have larger fmailies. And also, as parnets get older, their children won't be cosidred as dependents anymore. So it makes sense to have more smaples with fewer children.
-The skewness of Charges indicates that there are few people who are being charged higher than average. This may lead to some bias in the study. -The skwness and non-normal distibution of these categories is partly responsible for the low correlation we find between them

In [ ]:
# Visualizing the frequency of the smoker category:
counts = ins['smoker'].value_counts() 
counts.plot.bar(color = 'blue', grid=False) 

In [ ]:
#For the numeric data, we can compare them with the combined histogram and KDNs plot:
def plot_density_hist(ins, cols, bins = 10, hist = False):
    for col in cols:
        sns.set_style("whitegrid")
        sns.distplot(ins[col], bins = bins, rug=True, hist = hist)
        plt.title('Histogram of ' + col) 
        plt.xlabel(col) 
        plt.ylabel('')
        plt.show()
        
plot_cols = ['bmi', 'charges', 'age', 'children']
plot_density_hist(ins, plot_cols, bins = 20, hist = True) 

The data is much clearer now. (Notice how all the data is normalized on the y axis such that the area under the plot is equal to 1.)
1- Notice how the bmi is fairly normal with few outliers to the right, but not much to make a noticable skew.
2- We knew that the charges were right-skewed but now we can see a samll bump between 30000 and 40000$ whcih we may need to investigate further.
3- Again, the number of participants around the age of 20 is too large compared to the rest. this could be the reason why the children plot is very much skewd to the right and has no bell shape at all.

# Visualizing the frequency of the smoker category:(smoking or no);

In [ ]:
#Checking the skewness of charges
skew = ss.skewtest(ins['charges'])
skew

In [ ]:
def plot_scatter(ins, cols, col_y = 'charges'):
    for col in cols:
        fig = plt.figure(figsize=(7,6)) # define plot area
        ax = fig.gca() # define axis   
        ins.plot.scatter(x = col, y = col_y, ax = ax)
        ax.set_title('Scatter plot of ' + col_y + ' vs. ' + col) 
        ax.set_xlabel(col) 
        ax.set_ylabel(col_y)
        plt.show()

num_cols = ['age', 'bmi', 'children']
plot_scatter(ins, num_cols)   

The above plots suffer from a some over-plotting of the data, especially the bmi-charges one. To resolve this we can use transparency in the data points or use a different type of plot, the contour plot

# secondly i will provide the Statistical analysis concept to the data******

# #Describing the Statistical Properties of the Data

In [ ]:

ins.describe().round(3)

# We now compare charges by grouping them by the smoker category and look at the mean and std.

In [ ]:

charges_grouped = ins[['charges','smoker']].groupby('smoker')
print(' Mean by smoker')
print(charges_grouped.mean().round(2))
print('\n Standard deviation by smoker')
print(charges_grouped.std().round(2))

It is obvious how much the smoker and non-smoker differ in their medical charges, and also by their standard deviation; std of smokers is more than double of that of non-smoker. Hence while non-smokers seem to have their medical charges varying little about the mean, in other words, their medical health seems to be, more or less, similar. Non-smokers, on the other hand, vary a lot around the mean, which indicates that their bodies respond differently to smoking (e.g. some may develop lung cancer while others won't)

We now run a 2-tailed t-test to test whether the difference we see above in the smoking categeory could be due to chance, in other words, due to sampling error (H0) or is it a real difference (H1)

# **#Finally  Classification**
Now I do a classification model to predict whether someone is a smoker or not from his insurance data. 
First, we start by digitizing the smoker category, however not in a binary code, but in single digit for each subcategory, e.g. 1 for Yes and 0 for no. This is so we won't get a double entry into the algorithm model. This is why I don't use the one-hot-encoder here.

In [ ]:
label = np.array(ins['smoker'])
enc = preprocessing.LabelEncoder()
enc.fit(label)
label = enc.transform(label)
label.shape

In [ ]:
#We proceed as before.
def encode_string(cat_features):
    enc = preprocessing.LabelEncoder()
    enc.fit(cat_features)
    enc_cat_features = enc.transform(cat_features)
    ohe = preprocessing.OneHotEncoder()
    encoded = ohe.fit(enc_cat_features.reshape(-1,1))
    return encoded.transform(enc_cat_features.reshape(-1,1)).toarray()

categorical_columns = ['sex']

Features = encode_string(ins['region'])
for col in categorical_columns:
    temp = encode_string(ins[col])
    Features = np.concatenate([Features, temp], axis = 1)

Features = np.concatenate([Features, np.array(ins[['bmi', 'charges','children', 'age']])], axis = 1)
print(Features.shape)
print(Features[:2, :])

In [ ]:
nr.seed(1144)
indx = range(Features.shape[0])
indx = ms.train_test_split(indx, test_size = 200)
x_train = Features[indx[0],:]
y_train = np.ravel(label[indx[0]])
x_test = Features[indx[1],:]
y_test = np.ravel(label[indx[1]])
x_train.shape

In [ ]:
#Scaling the numeric values
scaler = preprocessing.StandardScaler().fit(x_train[:,6:])
x_train[:,6:] = scaler.transform(x_train[:,6:])
x_test[:,6:] = scaler.transform(x_test[:,6:])
x_train[:2]

We now build the non-linear model. Notice that I included a class-weight correction to take into consideration that the number of smokers is much less than nonsmoker

In [ ]:
logistic_mod = linear_model.LogisticRegression(fit_intercept = False, class_weight={0:0.8, 1:0.2}) 
logistic_mod.fit(x_train, y_train)
print(logistic_mod.intercept_)
print(logistic_mod.coef_)

Below are the probabilities of smoker category, yes or no, using the logistic function.

In [ ]:
probabilities = logistic_mod.predict_proba(x_test)
print(probabilities[:5,:])

Now we set a threshold value to turn the above probabilities into binary digits of 1 and 0 to compare with the test values. We chose a threshold of 0.5.

In [ ]:
def score_model(probs, threshold):
    return np.array([1 if x > threshold else 0 for x in probs[:,1]])
scores = score_model(probabilities, 0.5)
print(np.array(scores[:100]))
print(y_test[:100])

In [ ]:
#We set up a confusion table to examine our scores:
def print_metrics(label, scores):
    metrics = sklm.precision_recall_fscore_support(label, scores)
    conf = sklm.confusion_matrix(label, scores)
    print('                 Confusion matrix')
    print('                 Score positive    Score negative')
    print('Actual positive    %6d' % conf[0,0] + '             %5d' % conf[0,1])
    print('Actual negative    %6d' % conf[1,0] + '             %5d' % conf[1,1])
    print('')
    print('Accuracy  %0.2f' % sklm.accuracy_score(label, scores))
    print(' ')
    print('           Positive      Negative')
    print('Num case   %6d' % metrics[3][0] + '          %6d' % metrics[3][1])
    print('Precision  %6.2f' % metrics[0][0] + '        %6.2f' % metrics[0][1])
    print('Recall     %6.2f' % metrics[1][0] + '        %6.2f' % metrics[1][1])
    print('F1         %6.2f' % metrics[2][0] + '        %6.2f' % metrics[2][1])
   
print_metrics(y_test, scores) 

In [ ]:
#Plotting the ROC curve
def plot_auc(label, probs):
    fpr, tpr, threshold = sklm.roc_curve(label, probs[:,1])
    auc = sklm.auc(fpr, tpr)
        
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, color = 'orange', label = 'AUC = %0.2f' % auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()
    
plot_auc(y_test, probabilities)  

The accuracy of the classification model is 94% and of the 200 test values, the model only miss-labeled 13 of them. This is also obvious from the ROC curve with an area of 99%.
This seems too good to be true, if it is not for the fact that, as we saw earlier, smoker and nonsmoker values are very much separated. This makes it much easier for the system to predict them correctly.
Nevertheless, the model seems to have missed a big chunk of the smoker category only 66% accurate.
Interestingly, when I ran the same model but without class_weight, the results were much better, only 5 missed from the non-smoker category and with accuracy of 99%. So, I guess when the two classification labels are very much separated, maybe it is better not to use weight correction.
That been said, a new set of data that may have a larger overlapping between the smoker two cases, may not be predicted as well as